<a href="https://colab.research.google.com/github/Brunapupo/llama2/blob/main/dataset_classifica%C3%A7%C3%A3o_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import nltk
import nltk.corpus
from nltk.stem import RSLPStemmer
from nltk.tokenize import sent_tokenize
from string import punctuation
from collections import defaultdict
from heapq import nlargest
from nltk.tokenize import word_tokenize, sent_tokenize
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer
from nltk.corpus import floresta
from nltk.corpus import machado
from bs4 import BeautifulStoneSoup, BeautifulSoup
import cgi

In [1]:
# Montar o Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Definir o caminho do arquivo no Google Drive
file_path = '/content/drive/MyDrive/Documentos-stt/Teleconsultorias-anonimizadas-v1 - Teleconsultorias-anonimizadas-v1 (1).csv'

# Carregar o arquivo usando pandas
import pandas as pd

df = pd.read_csv(file_path)
df.head()  # Visualizar as primeiras linhas do dataframe


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Unnamed: 0,pergunta,resposta,assunto,tipo,classificacao,especificacao,detalhamento,cid10,satisfacao_geral,...,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27
0,0,"""em anexo \ncurativo [|PACIENTE|] a [|PACIENT...","""boa tarde enfª [|PACIENTE|] y, \n\n\n\nanexo ...","""curativo [|PACIENTE|] a [|PACIENTE|] [|PACIE...","""assíncrona""","""dúvidas clínicas gerais""","""s - pele""","""56 - curativo/bandagem/compressão/tamponamento""","""""","""""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13,[|PACIENTE|] teve febre reumática aos 5 anos ...,"""a duração da profilaxia depende da idade do p...","""profilaxia febre reumática""","""assíncrona""","""dúvidas clínicas gerais""","""k - aparelho circulatório""","""99 - outras doenças do aparelho circulatório""","""""","""insatisfeito(a)""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,21,"""criança com 10 meses, refere episodio de aler...","""thalia, por ser cultivada em ovos embrionados...","""alergia a ovo""","""assíncrona""","""dúvidas clínicas gerais""","""a - geral e não-específico""","""92 - alergia/reação alérgica ne""","""""","""satisfeito(a)""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,37,"""26 anos\n\n#consulta médica\n\n#tabagista 10c...","""raisa, o ministério da saúde preconiza que os...","""gravidez ectópica x anticoncepcional""","""assíncrona""","""dúvidas clínicas gerais""","""w - gravidez e planejamento familiar""","""80 - gravidez ectópica""","""""","""satisfeito(a)""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,42,"""se há necessidade de revisão e/ou acompanhame...","""prezado colega\n\na manutenção da fav em paci...","""fistula arterio venosa""","""assíncrona""","""dúvidas clínicas gerais""","""u - aparelho urinário""","""67 - referenciado para médico/especialista/cl...","""""","""""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
import pandas as pd

# Montar o Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Definir o caminho dos arquivos CSV
capitulos_path = '/content/drive/MyDrive/Documentos-stt/Dicionário-de-termos/CID-10-CAPITULOS.CSV'
categorias_path = '/content/drive/MyDrive/Documentos-stt/Dicionário-de-termos/CID-10-CATEGORIAS.CSV'
grupos_path = '/content/drive/MyDrive/Documentos-stt/Dicionário-de-termos/CID-10-GRUPOS.CSV'
subcategorias_path = '/content/drive/MyDrive/Documentos-stt/Dicionário-de-termos/CID-10-SUBCATEGORIAS.CSV'

# Carregar os arquivos CSV com codificação 'latin1' e delimitador ';'
capitulos_df = pd.read_csv(capitulos_path, encoding='latin1', delimiter=';')
categorias_df = pd.read_csv(categorias_path, encoding='latin1', delimiter=';')
grupos_df = pd.read_csv(grupos_path, encoding='latin1', delimiter=';')
subcategorias_df = pd.read_csv(subcategorias_path, encoding='latin1', delimiter=';')

# Extrair termos relevantes para o dicionário
termos_cid = set()

# Adicionar termos de cada dataframe ao conjunto
termos_cid.update(capitulos_df['DESCRICAO'].str.lower().tolist())
termos_cid.update(categorias_df['DESCRICAO'].str.lower().tolist())
termos_cid.update(grupos_df['DESCRICAO'].str.lower().tolist())
termos_cid.update(subcategorias_df['DESCRICAO'].str.lower().tolist())

# Remover possíveis NaNs
termos_cid = {termo for termo in termos_cid if isinstance(termo, str)}

# Visualizar alguns termos
print(list(termos_cid)[:20])

# Salvar os termos em um novo arquivo CSV
termos_cid_df = pd.DataFrame(list(termos_cid), columns=['Termo'])
termos_cid_df.to_csv('/content/drive/MyDrive/Documentos-stt/Dicionário-de-termos/termos_cid.csv', index=False)

print("Dicionário de termos salvo em /content/drive/MyDrive/Documentos-stt/Dicionário-de-termos/termos_cid.csv")


Mounted at /content/drive


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


['presbiacus', 'necess imuniz contr algum doenç viral únic', 'rickettsios especific', 'outr dificuldad especific relacion facil médic outr cuid saúd', 'atax cerebel iníci precoc', 'alopéc mucin', 'histór famili doenç pel tec subcutâne', 'compromet pálpebr outr doenç classific outr part', 'pneumon doenç bacteri classific outr part', 'esmagado, empurr pisote mult deband mass pesso rua estr', 'osteocondrodisplas especific anomal cresc oss long colun vertebr', 'amput traumá pern nível especific', 'outr exam geral', 'outr form hipertens secundár', 'artrit reumatóid juvenil', 'ocup caminhonet traum colis tr [comboio] veícul ferrovi condu [motorista] traum acid trânsit', 'pesso esper admit estabelec médic adequ outr part', 'dispar outr arm fog arm fog especificada, intenç determin áre comérci serviç', 'neoplas malign esôfag', 'rifle, espingard arm fog mai tamanh fazend']
Dicionário de termos refinado salvo em /content/drive/MyDrive/Documentos-stt/Dicionário-de-termos/termos_cid_refinado.csv


In [23]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string

# Montar o Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Definir o caminho dos arquivos CSV
capitulos_path = '/content/drive/MyDrive/Documentos-stt/Dicionário-de-termos/CID-10-CAPITULOS.CSV'
categorias_path = '/content/drive/MyDrive/Documentos-stt/Dicionário-de-termos/CID-10-CATEGORIAS.CSV'
grupos_path = '/content/drive/MyDrive/Documentos-stt/Dicionário-de-termos/CID-10-GRUPOS.CSV'
subcategorias_path = '/content/drive/MyDrive/Documentos-stt/Dicionário-de-termos/CID-10-SUBCATEGORIAS.CSV'

# Carregar os arquivos CSV com codificação 'latin1' e delimitador ';'
capitulos_df = pd.read_csv(capitulos_path, encoding='latin1', delimiter=';')
categorias_df = pd.read_csv(categorias_path, encoding='latin1', delimiter=';')
grupos_df = pd.read_csv(grupos_path, encoding='latin1', delimiter=';')
subcategorias_df = pd.read_csv(subcategorias_path, encoding='latin1', delimiter=';')

# Inicializar stop words
nltk.download('stopwords')
stop_words = set(stopwords.words('portuguese'))

# Função para pré-processar os termos
def preprocess_term(term):
    # Tokenizar e remover pontuação
    tokens = [word.strip(string.punctuation) for word in term.split()]
    # Remover stop words
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    return " ".join(filtered_tokens) if filtered_tokens else None

# Extrair e pré-processar termos relevantes para o dicionário
termos_cid = set()

# Adicionar termos de cada dataframe ao conjunto
termos_cid.update(capitulos_df['DESCRICAO'].dropna().str.lower().apply(preprocess_term).dropna().tolist())
termos_cid.update(categorias_df['DESCRICAO'].dropna().str.lower().apply(preprocess_term).dropna().tolist())
termos_cid.update(grupos_df['DESCRICAO'].dropna().str.lower().apply(preprocess_term).dropna().tolist())
termos_cid.update(subcategorias_df['DESCRICAO'].dropna().str.lower().apply(preprocess_term).dropna().tolist())

# Remover possíveis termos vazios
termos_cid = {termo for termo in termos_cid if termo.strip()}

# Visualizar alguns termos
print(list(termos_cid)[:20])

# Salvar os termos em um arquivo TXT
with open('/content/drive/MyDrive/Documentos-stt/Dicionário-de-termos/termos_cid.txt', 'w', encoding='utf-8') as f:
    for termo in termos_cid:
        f.write(f"{termo}\n")

print("Dicionário de termos salvo em /content/drive/MyDrive/Documentos-stt/Dicionário-de-termos/termos_cid.txt")


Mounted at /content/drive
['ocupante triciclo motorizado traumatizado acidente transporte colisão  condutor traumatizado acidente não-de-trânsito', 'encurvamento congênito ossos longos especificados membro inferior', 'calos calosidades', 'auto-intoxicação exposição intencional outras drogas medicamentos substâncias biológicas especificadas', 'outras quedas nível  áreas comércio serviços', 'tromboangeíte obliterante doença buerger', 'traumatismo nervo abducente', 'outras formas carbúnculo', 'ocupante veículo transporte pesado traumatizado colisão veículo pedal  ocupante especificado veículo transporte pesado traumatizado acidente não-de-trânsito', 'motociclista traumatizado colisão veículo transporte pesado ônibus  passageiro traumatizado acidente trânsito', 'hiperparatireoidismo especificado', 'contusão pálpebra região periocular', 'agressão meio disparo arma fogo mão  fazenda', 'impacto acidental ativo passivo causado equipamento esportivo  áreas comércio serviços', 'escoliose toracog

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string

# Montar o Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Definir o caminho dos arquivos CSV
capitulos_path = '/content/drive/MyDrive/Documentos-stt/Dicionário-de-termos/CID-10-CAPITULOS.CSV'
categorias_path = '/content/drive/MyDrive/Documentos-stt/Dicionário-de-termos/CID-10-CATEGORIAS.CSV'
grupos_path = '/content/drive/MyDrive/Documentos-stt/Dicionário-de-termos/CID-10-GRUPOS.CSV'
subcategorias_path = '/content/drive/MyDrive/Documentos-stt/Dicionário-de-termos/CID-10-SUBCATEGORIAS.CSV'

# Carregar os arquivos CSV com codificação 'latin1' e delimitador ';'
capitulos_df = pd.read_csv(capitulos_path, encoding='latin1', delimiter=';')
categorias_df = pd.read_csv(categorias_path, encoding='latin1', delimiter=';')
grupos_df = pd.read_csv(grupos_path, encoding='latin1', delimiter=';')
subcategorias_df = pd.read_csv(subcategorias_path, encoding='latin1', delimiter=';')

# Inicializar stop words
nltk.download('stopwords')
stop_words = set(stopwords.words('portuguese'))

# Função para pré-processar os termos
def preprocess_term(term):
    # Tokenizar e remover pontuação
    tokens = [word.strip(string.punctuation) for word in term.split()]
    # Remover stop words
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    return " ".join(filtered_tokens) if filtered_tokens else None

# Extrair e pré-processar termos relevantes para o dicionário
termos_cid = set()

# Adicionar termos de cada dataframe ao conjunto
termos_cid.update(capitulos_df['DESCRICAO'].dropna().str.lower().apply(preprocess_term).dropna().tolist())
termos_cid.update(categorias_df['DESCRICAO'].dropna().str.lower().apply(preprocess_term).dropna().tolist())
termos_cid.update(grupos_df['DESCRICAO'].dropna().str.lower().apply(preprocess_term).dropna().tolist())
termos_cid.update(subcategorias_df['DESCRICAO'].dropna().str.lower().apply(preprocess_term).dropna().tolist())

# Remover possíveis termos vazios
termos_cid = {termo for termo in termos_cid if termo.strip()}

# Visualizar alguns termos
print(list(termos_cid)[:20])

# Salvar os termos em um arquivo TXT
with open('/content/drive/MyDrive/Documentos-stt/Dicionário-de-termos/termos_cid_refinado.txt', 'w', encoding='utf-8') as f:
    for termo in termos_cid:
        f.write(f"{termo}\n")

print("Dicionário de termos refinado salvo em /content/drive/MyDrive/Documentos-stt/Dicionário-de-termos/termos_cid_refinado.txt")


Mounted at /content/drive
['ocupante triciclo motorizado traumatizado acidente transporte colisão  condutor traumatizado acidente não-de-trânsito', 'encurvamento congênito ossos longos especificados membro inferior', 'calos calosidades', 'auto-intoxicação exposição intencional outras drogas medicamentos substâncias biológicas especificadas', 'outras quedas nível  áreas comércio serviços', 'tromboangeíte obliterante doença buerger', 'traumatismo nervo abducente', 'outras formas carbúnculo', 'ocupante veículo transporte pesado traumatizado colisão veículo pedal  ocupante especificado veículo transporte pesado traumatizado acidente não-de-trânsito', 'motociclista traumatizado colisão veículo transporte pesado ônibus  passageiro traumatizado acidente trânsito', 'hiperparatireoidismo especificado', 'contusão pálpebra região periocular', 'agressão meio disparo arma fogo mão  fazenda', 'impacto acidental ativo passivo causado equipamento esportivo  áreas comércio serviços', 'escoliose toracog

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# **2- Extrair Termos Relevantes e Criar um Dicionário de Tokens**
Vamos extrair os termos da coluna DESCRICAO e criar um conjunto de tokens:

In [16]:
# Extrair termos relevantes para o dicionário
termos_cid = set()

# Adicionar termos da coluna 'DESCRICAO' ao conjunto
termos_cid.update(categorias_df['DESCRICAO'].str.lower().tolist())

# Remover possíveis NaNs
termos_cid = {termo for termo in termos_cid if isinstance(termo, str)}

# Visualizar alguns termos
print(list(termos_cid)[:10])


['diarréia e gastroenterite de origem infecciosa presumível', 'transtornos fóbico-ansiosos', 'luxação, entorse e distensão de articulações e dos ligamentos do tórax', 'exame médico e consulta com finalidades administrativas', 'traumatismo de outros órgãos intratorácicos e dos não especificados', 'valor anormal da pressão arterial sem diagnóstico', 'doenças infecciosas e parasitárias maternas classificáveis em outra parte mas que compliquem a gravidez, o parto e o puerpério', 'transtornos de músculo em doenças classificadas em outra parte', 'viagem e movimento', 'neoplasia maligna das meninges']


# **stop_words:** Carrega a lista de stopwords em português e adiciona caracteres de pontuação e outras strings irrelevantes à lista.

In [5]:
import nltk

# Faça o download dos recursos necessários do NLTK
nltk.download('punkt')
nltk.download('rslp')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('floresta')
nltk.download('mac_morpho')
nltk.download('machado')
nltk.download('wordnet')
nltk.download('words')

# Inicializa o stemmer para português
stemmer = RSLPStemmer()
# Carrega a lista de stopwords em português
stop_words = nltk.corpus.stopwords.words('portuguese')
# Adiciona pontuação à lista de stopwords
stop_words = stop_words + list(punctuation)
stop_words = stop_words + ['\'\'', '\"\"', '\'', '\"', '``', '`', '\\n', 'p', '/p',
                           'ccedil', 'aacute','eacute','oacute','uacute', 'iacute', 'nbsp', 'br', 'span', '/span', 'otilde', 'atilde', 'div', '/div' ]
# Converte a lista de stopwords para um conjunto (para busca rápida)
stop_words_set = set(stop_words)

class text_color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package floresta to /root/nltk_data...
[nltk_data]   Package floresta is already up-to-date!
[nltk_data] Downloading package mac_morpho to /root/nltk_data...
[nltk_data]   Package mac_morpho is already up-to-date!
[nltk_data] Downloading package machado to /root/nltk_data...
[nltk_data]   Package machado is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk

# **Passo 2: Importar SpaCy e Carregar o Modelo**

In [7]:
# Instalar o SpaCy e o modelo de linguagem portuguesa
!pip install spacy
!python -m spacy download pt_core_news_sm

import spacy
import pt_core_news_sm

nlp = pt_core_news_sm.load()

# Teste de processamento de texto
doc = nlp("nódulo tireoidiano")
print([(w.text, w.pos_, w.lemma_) for w in doc])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 64.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
[('nódulo', 'ADJ', 'nódulo'), ('tireoidiano', 'ADJ', 'tireoidiano')]


In [8]:
doc = nlp("nodulo tireoidiano")
print([(w.text, w.pos_, w.lemma_) for w in doc])


[('nodulo', 'NOUN', 'nodulo'), ('tireoidiano', 'ADJ', 'tireoidiano')]


In [9]:
for token in "nódulo de tireóide".split():
    print(stemmer.stem(token))

nódul
de
tireóid


In [10]:
type(stop_words)


list

In [11]:
lemmatizer = WordNetLemmatizer()
print(lemmatizer.lemmatize("poliartralgia"))


poliartralgia


In [12]:
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------


KeyboardInterrupt: Interrupted by user